<a href="https://colab.research.google.com/github/A-l-Mahi/motion-prediction/blob/main/Analysis_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount and authenticate Drive
library("googledrive")
drive_auth(
  email = gargle::gargle_oauth_email(),
  path = NULL,
  scopes = "https://www.googleapis.com/auth/drive",
  cache = gargle::gargle_oauth_cache(),
  use_oob = gargle::gargle_oob_default(),
  token = NULL
)

Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No


Selection: yes


Enter a number between 1 and 2, or enter 0 to exit.


Selection: 1


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=ecfc45d770a18435e25085e98d1a5fbe&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWRldTVCV0ZpY1hyTkFreU9LbU5CSEVjNEZqcG9WRG5rZU4wLWtYZWx3blRBaG1rb1ZQd21TT0JhbkdvMTM5MGJGYVZTUSIsInN0YXRlIjoiZWNmYzQ1ZDc3MGExODQzNWUyNTA4NWU5OGQxYTVmYmUifQ==


In [ ]:
install.packages(c("gbm", "randomForest", "caret")); install.packages("randomForest");install.packages("caret")

library(dplyr); library(caret); library(ggplot2); require(parallel); require(dplyr)

set.seed(333)

In [3]:
drive_download("pml-testing.csv", type = "csv");drive_download("pml-training.csv", type = "csv")

! Ignoring `type`. Only consulted for native Google file types.

  MIME type of `file`: mime_type.

File downloaded:

• pml-testing.csv <id: 1_m9q2fS08KrE4F3WmAVYrgVDp2-3TqDm>

Saved locally as:

• pml-testing.csv

! Ignoring `type`. Only consulted for native Google file types.

  MIME type of `file`: mime_type.

File downloaded:

• pml-training.csv <id: 1QyUzByGNMruYYAjIz9qpk8zCMSSC8VKX>

Saved locally as:

• pml-training.csv



In [4]:
pml <- read.csv("pml-training.csv")
length(names(pml))

[1] 160

In [5]:
# preProcessing
# remove the NAs of the data.

#Now lets check for NAs

#Here we get the percentage of variables that are Na or empty characters
pml$classe <- as.factor(pml$classe)
df <-  pml[, colSums(is.na(pml)) == 0] # there seems to be some columns are all NAs

df <- df[,colSums(df == "") == 0] # Here we check for empty characters

1 - (length(names(df)) / length(names(pml)))

# now we can see that 62.5 of the data are either Nas of Empty characters

# we are now down to 60 columns

df <- df[, -c(which(grepl("^X|timestamp|window|user_name", names(df))))]

# removing less meaningful variables X|timestamp|window|user_name

# And lastly 53 columns

[1] 0.625

In [6]:

# Each accelerometer have 13 readings that indicates it's positions on the volunteers.
# Reducing the data set to 50 observation due to limited computing power.
sam <- sample(nrow(pml), 500)

df <- df[sam ,]


# Considering the enough data set we have, lets slice the training data set to subTrain and validation set


subTrain <- createDataPartition(y = df$classe, p = 0.60, list = F)

subTraining <- df[subTrain, ]
validation <- df[-subTrain, ]


#PCA on subTraining

#Pricinple Component Analysis


subTraining[, -53] <- lapply(subTraining[, -53], as.numeric)

#corSub <- cor(subTraining[, -53])

#summary(corSub[upper.tri(corSub)])

#highlyCor <- findCorrelation(corSub, cutoff = .9)

#corTraining <- subTraining[, -highlyCor]

#corTraining[, -53] <- lapply(corTraining[, -53], as.numeric)

#corSub2 <- cor(corTraining[, -53])

#summary(corSub[upper.tri(corSub2)])




In [ ]:
fitControl <- trainControl(
  method = "repeatedcv",
  number = 10,
  repeats = 10,
  classProbs = T
  )

In [ ]:
#Here we have fit 4 models Logistic Regression, Decision Trees, Random Forest and K-Nearest Neighbour respectively

modFitGBM <- train(classe ~ ., method = "gbm", data = subTraining,
          trControl = fitControl
)
modFitFor <- train(classe ~., method = "rf", data = subTraining,
          trContol = fitControl
)

modFitNb <- train(classe ~., method = "nb", data = subTraining,
          trControl = fitControl
)


In [ ]:
preLogit <- predict(modFitGBM, newdata = validation)
preFor <- predict(modFitFor, newdata = validation)
preNb <- predict(modFitNb,newdata = validation)

In [ ]:
confusionMatrix(preLogit, validation$classe)
confusionMatrix(preFor, validation$classe)
confusionMatrix(preNb, validation$classe)

In [ ]:
preTree <- predict(modFitTree, newdata = validation)

In [ ]:
confusionMatrix(preTree, validation$classe)

In [ ]:
list.dirs("content/drive")

character(0)